In [1]:
from datasets import load_dataset
squad = load_dataset("squad", split="train[:5000]")


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

plain_text/validation-00000-of-00001.par(…):   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [5]:
squad = squad.train_test_split(test_size=0.2)
squad["train"][0]

{'id': '5733bed24776f4190066118b',
 'title': 'University_of_Notre_Dame',
 'context': 'The university is the major seat of the Congregation of Holy Cross (albeit not its official headquarters, which are in Rome). Its main seminary, Moreau Seminary, is located on the campus across St. Joseph lake from the Main Building. Old College, the oldest building on campus and located near the shore of St. Mary lake, houses undergraduate seminarians. Retired priests and brothers reside in Fatima House (a former retreat center), Holy Cross House, as well as Columba Hall near the Grotto. The university through the Moreau Seminary has ties to theologian Frederick Buechner. While not Catholic, Buechner has praised writers from Notre Dame and Moreau Seminary created a Buechner Prize for Preaching.',
 'question': 'What individuals live at Fatima House at Notre Dame?',
 'answers': {'text': ['Retired priests and brothers'], 'answer_start': [356]}}

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [8]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length = 384, 
        truncation = "only_second", 
        return_offsets_mapping = True, 
        padding = "max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_pos = []
    end_pos = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids()

        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx -1
    
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_pos.append(0)
            end_pos.append(0)
        else:
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx +=1
            start_pos.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][0] <= end_char:
                idx -=1
            end_pos.append(idx + 1)
    inputs["start_positions"] = start_pos
    inputs["end_positions"] = end_pos
    return inputs


In [9]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [10]:
from transformers import DefaultDataCollator
data_collator = DefaultDataCollator()

In [11]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
model = AutoModelForQuestionAnswering.from_pretrained("distilbert/distilbert-base-uncased")

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
training_args = TrainingArguments(
    output_dir = "first_qa_model",
    eval_strategy = "epoch",
    learning_rate= 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    num_train_epochs = 3,
    weight_decay = 0.01,
    push_to_hub = True,
)


trainer = Trainer(
    model = model, 
    args = training_args,
    train_dataset = tokenized_squad["train"],
    eval_dataset = tokenized_squad["test"],
    processing_class = tokenizer,
    data_collator = data_collator, 
)

trainer.train()

/Users/pranamshetty/Developer/workshop/transformers-from-scratch/venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss
1,No log,1.771555
2,2.014300,2.548612
3,2.014300,2.650872


/Users/pranamshetty/Developer/workshop/transformers-from-scratch/venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/pranamshetty/Developer/workshop/transformers-from-scratch/venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=750, training_loss=1.7734112548828125, metrics={'train_runtime': 525.0316, 'train_samples_per_second': 22.856, 'train_steps_per_second': 1.428, 'total_flos': 1175877900288000.0, 'train_loss': 1.7734112548828125, 'epoch': 3.0})